## Colab & Drive環境設定

### Drive連結

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


### 切換路徑

In [1]:
cd drive/My Drive/PhD/程式/TrajectoryPrediction/STGAT 

/content/drive/My Drive/PhD/程式/TrajectoryPrediction/STGAT


## 主程式

### Library import & Parameters

In [2]:
""" 
# 先從STGAT引用package # 
"""
import sys 
sys.path.append('/drive/My Drive/PhD/程式/TrajectoryPrediction/STGAT')

from STGAT import __init__
from STGAT import trajectories
from STGAT import loader
from STGAT import draw_trajectory
from STGAT import evaluate_model

from STGAT import models
from STGAT import train
from STGAT import utils

import argparse
import logging
import os
import random
import shutil
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

import STGAT.utils
from STGAT.loader import data_loader
from STGAT.models import TrajectoryGenerator
from STGAT.utils import (
    displacement_error,
    final_displacement_error,
    get_dset_path,
    int_tuple,
    l2_loss,
    relative_to_abs,
)

import easydict
args = easydict.EasyDict({
        "log_dir": "./",
        "dataset_name": "zara2",
        "delim": "\t",
        "loader_num_workers": 1,  #原:4
        "obs_len": 8,
        "pred_len": 12,
        "skip": 1,
        "seed": 72,
        "batch_size":64,
        
        "noise_dim":(16,),
        "noise_type":"gaussian",
        "traj_lstm_input_size":2,
        "traj_lstm_hidden_size":32,
        "heads":"4,1",
        "hidden_units":"16",
        "graph_network_out_dims":32,
        "graph_lstm_hidden_size":32,
        "dropout":0,
        "alpha": 0.2,
        "lr":1e-3,
        "start_epoch":0,
        "best_k":20,
        "print_every":10,
        "use_gpu":1,
        "gpu_num":"0",
        "resume":"checkpoint/test0809_1",

        "num_epochs":1,

        "min_ped_in_scene" :1  # control the minimum number of pedestrain 
})

best_ade = 100

## Main Run

In [3]:
import sys
def main(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu_num
    train_path = get_dset_path(args.dataset_name, "train")
    val_path = get_dset_path(args.dataset_name, "test")

    print("$Colad_Train.ipynb","call data_loader()")
    train_dset, train_loader = data_loader(args, train_path)
    
    '''
    _, val_loader = data_loader(args, val_path)
    '''
    writer = SummaryWriter()
    
    # n_units = [32, 16, 32]
    n_units = (
        [args.traj_lstm_hidden_size]      #traj_lstm_hidden_size=32
        + [int(x) for x in args.hidden_units.strip().split(",")]  #hidden_units=16
        + [args.graph_lstm_hidden_size]   #graph_lstm_hidden_size=32
    )
    print("$Colad_Train.ipynb","-n_units:",n_units)
   
    n_heads = [int(x) for x in args.heads.strip().split(",")]
    print("$Colad_Train.ipynb","-n_heads:",n_heads)
    
    model = TrajectoryGenerator(
        obs_len=args.obs_len,
        pred_len=args.pred_len,
        traj_lstm_input_size=args.traj_lstm_input_size, #2
        traj_lstm_hidden_size=args.traj_lstm_hidden_size, #32
        n_units=n_units, #[32, 16, 32]
        n_heads=n_heads, #[4,1]
        graph_network_out_dims=args.graph_network_out_dims,  #32
        dropout=args.dropout,
        alpha=args.alpha,
        graph_lstm_hidden_size=args.graph_lstm_hidden_size,   #32
        noise_dim=args.noise_dim,
        noise_type=args.noise_type,
    )
    
    print("$Colad_Train.ipynb","Model.summary:\n",model)
    
    model.cuda()  # cuda()函數能實現從CPU到GPU的内存遷移
    optimizer = optim.Adam(
        [
            {"params": model.traj_lstm_model.parameters(), "lr": 1e-2},
            {"params": model.traj_hidden2pos.parameters()},
            {"params": model.gatencoder.parameters(), "lr": 3e-2},
            {"params": model.graph_lstm_model.parameters(), "lr": 1e-2},
            {"params": model.traj_gat_hidden2pos.parameters()},
            {"params": model.pred_lstm_model.parameters()},
            {"params": model.pred_hidden2pos.parameters()},
        ],
        lr=args.lr,
    )
    global best_ade
    if args.resume:
        if os.path.isfile(args.resume):
            logging.info("Restoring from checkpoint {}".format(args.resume))
            checkpoint = torch.load(args.resume)
            args.start_epoch = checkpoint["epoch"]
            model.load_state_dict(checkpoint["state_dict"])
            logging.info(
                "=> loaded checkpoint '{}' (epoch {})".format(
                    args.resume, checkpoint["epoch"]
                )
            )
        else:
            logging.info("=> no checkpoint found at '{}'".format(args.resume))

    training_step = 1
    for epoch in range(args.start_epoch, args.num_epochs + 1):

        if epoch < 150:
            training_step = 1

        elif epoch < 250:
            training_step = 2

        else:
            if epoch == 250:
                for param_group in optimizer.param_groups:
                    param_group["lr"] = 5e-3
            training_step = 3

        if epoch >1:
            training_step = 3

        train(args, model, train_loader, optimizer, epoch, training_step, writer)
        

        if training_step == 3:

            print("@GN","training_step=3")
            ade = validate(args, model, val_loader, epoch, writer)
            is_best = ade < best_ade
            best_ade = min(ade, best_ade)

            print("@GN","save_checkpoint:")
            save_checkpoint(
                {
                    "epoch": epoch + 1,
                    "state_dict": model.state_dict(),
                    "best_ade": best_ade,
                    "optimizer": optimizer.state_dict(),
                },
                is_best,
                f"./checkpoint/checkpoint{epoch}.pth.tar",
            )        
    writer.close()

def train(args, model, train_loader, optimizer, epoch, training_step, writer):
    
    print("$Colad_Train.ipynb","call train()")
    losses = utils.AverageMeter("Loss", ":.6f")
    progress = utils.ProgressMeter(
        len(train_loader), [losses], prefix="Epoch: [{}]".format(epoch)
    )
    model.train()
    print("$Colad_Train.ipynb","model.train()")

    """
    $GN:
    - len(train_loader) == equals number of batches
    - ex: self.seq_start_end = 
        [(0, 2), (2, 4), (4, 7), (7, 10) ... (25499, 25501), (25501, 25503), (25503, 25505), (25505, 25507)]
        - length : 2112
        - each batch will take batch_size
        - ex: batch_1: [(0, 2), (2, 4), (4, 7), (7, 10)...(161, 164)] => 64個
              ...
              batch_n: [...(25503, 25505), (25505, 25507)]
        - so, train_loader.length = len(self.seq_start_end)/batch_size = 33
    """
    print("$Colad_Train.ipynb","train_loader.length", len(train_loader))
    for batch_idx, batch in enumerate(train_loader):
        print("$Colad_Train.ipynb","\nbatch_idx:",batch_idx, "\nbatch[0].shape:",batch[0].size())

        batch = [tensor.cuda() for tensor in batch]
        (
            obs_traj,
            pred_traj_gt,
            obs_traj_rel,
            pred_traj_gt_rel,
            non_linear_ped,
            loss_mask,
            seq_start_end,
        ) = batch

        print("$Colad_Train.ipynb","\nobs_traj.size():",obs_traj.size(), "\nobs_traj[0].shape:",obs_traj[0].size())

        optimizer.zero_grad()
        loss = torch.zeros(1).to(pred_traj_gt)
        l2_loss_rel = []
        loss_mask = loss_mask[:, args.obs_len :]

        if training_step == 1 or training_step == 2:
            model_input = obs_traj_rel
            print("$Colad_Train.ipynb","obs_traj_rel.size():",obs_traj_rel.size())
            
            pred_traj_fake_rel = model(
                model_input, obs_traj, seq_start_end, 1, training_step
            )
            l2_loss_rel.append(
                l2_loss(pred_traj_fake_rel, model_input, loss_mask, mode="raw")
            )

            
            print("$Colad_Train.ipynb","pred_traj_fake_rel:",pred_traj_fake_rel)
            
            
        else:
            print("@GN", "Training-step=3, ouput model")

            model_input = torch.cat((obs_traj_rel, pred_traj_gt_rel), dim=0)
            for _ in range(args.best_k):
                pred_traj_fake_rel = model(model_input, obs_traj, seq_start_end, 0)
                l2_loss_rel.append(
                    l2_loss(
                        pred_traj_fake_rel,
                        model_input[-args.pred_len :],
                        loss_mask,
                        mode="raw",
                    )
                )

        l2_loss_sum_rel = torch.zeros(1).to(pred_traj_gt)
        l2_loss_rel = torch.stack(l2_loss_rel, dim=1)
        for start, end in seq_start_end.data:
            _l2_loss_rel = torch.narrow(l2_loss_rel, 0, start, end - start)
            _l2_loss_rel = torch.sum(_l2_loss_rel, dim=0)  # [20]
            _l2_loss_rel = torch.min(_l2_loss_rel) / (
                (pred_traj_fake_rel.shape[0]) * (end - start)
            )
            l2_loss_sum_rel += _l2_loss_rel

        loss += l2_loss_sum_rel
        losses.update(loss.item(), obs_traj.shape[1])
        loss.backward()
        optimizer.step()
        if batch_idx % args.print_every == 0:
            progress.display(batch_idx)

    writer.add_scalar("train_loss", losses.avg, epoch)

def validate(args, model, val_loader, epoch, writer):
    ade = utils.AverageMeter("ADE", ":.6f")
    fde = utils.AverageMeter("FDE", ":.6f")
    progress = utils.ProgressMeter(len(val_loader), [ade, fde], prefix="Test: ")

    model.eval()
    with torch.no_grad():

        """
        $GN:
        - 1.執行for batch_idx, batch in enumerate(train_loader)
        - 2.DataLoader會依序抽出batch_size個dataset
            => 每次抽出會呼叫dataset內的__getitem__()得到out
        - 3.抽取完batch_size個dataset後, 呼叫自定義的方法
            => 傳入 [out_1,out_2, ... out_batch_size]
            
        """
        for i, batch in enumerate(val_loader):
            batch = [tensor.cuda() for tensor in batch]
            (
                obs_traj,
                pred_traj_gt,
                obs_traj_rel,
                pred_traj_gt_rel,
                non_linear_ped,
                loss_mask,
                seq_start_end,
            ) = batch
            loss_mask = loss_mask[:, args.obs_len :]
            pred_traj_fake_rel = model(obs_traj_rel, obs_traj, seq_start_end)

            pred_traj_fake_rel_predpart = pred_traj_fake_rel[-args.pred_len :]
            pred_traj_fake = relative_to_abs(pred_traj_fake_rel_predpart, obs_traj[-1])
            ade_, fde_ = cal_ade_fde(pred_traj_gt, pred_traj_fake)
            ade_ = ade_ / (obs_traj.shape[1] * args.pred_len)

            fde_ = fde_ / (obs_traj.shape[1])
            ade.update(ade_, obs_traj.shape[1])
            fde.update(fde_, obs_traj.shape[1])

            if i % args.print_every == 0:
                progress.display(i)

        logging.info(
            " * ADE  {ade.avg:.3f} FDE  {fde.avg:.3f}".format(ade=ade, fde=fde)
        )
        writer.add_scalar("val_ade", ade.avg, epoch)
    return ade.avg

def cal_ade_fde(pred_traj_gt, pred_traj_fake):
    ade = displacement_error(pred_traj_fake, pred_traj_gt)
    fde = final_displacement_error(pred_traj_fake[-1], pred_traj_gt[-1])
    return ade, fde

def save_checkpoint(state, is_best, filename="checkpoint.pth.tar"):
    if is_best:
        torch.save(state, filename)
        logging.info("-------------- lower ade ----------------")
        shutil.copyfile(filename, "model_best.pth.tar")

if __name__ == "__main__":
    #args = parser.parse_args()
    utils.set_logger(os.path.join(args.log_dir, "train.log"))
    checkpoint_dir = "./checkpoint"
    if os.path.exists(checkpoint_dir) is False:
        os.mkdir(checkpoint_dir)
    main(args)


$Colad_Train.ipynb call data_loader()
$loader.py call TrajectoryDataset()
$trajectories.py ----min_ped: 1
$trajectories.py #STEP-1 讀取檔案:  File_Path: /content/drive/My Drive/PhD/程式/TrajectoryPrediction/STGAT/STGAT/datasets/zara2/train/biwi_eth_train.txt
$trajectories.py #STEP-2 unique time-frame數量: 700
$trajectories.py #Step-3 相同time_frame的element合併
ex: data[]: (3666, 4) ->frame_data[] (700,)
$trajectories.py #Step-4 扣除obs_len+pre_len,剩餘可處理長度: 681
$trajectories.py 更新seq_list數量: 40
$trajectories.py #STEP-1 讀取檔案:  File_Path: /content/drive/My Drive/PhD/程式/TrajectoryPrediction/STGAT/STGAT/datasets/zara2/train/biwi_hotel_train.txt
$trajectories.py #STEP-2 unique time-frame數量: 934
$trajectories.py #Step-3 相同time_frame的element合併
ex: data[]: (4946, 4) ->frame_data[] (934,)
$trajectories.py #Step-4 扣除obs_len+pre_len,剩餘可處理長度: 915
$trajectories.py 更新seq_list數量: 271
$trajectories.py #STEP-1 讀取檔案:  File_Path: /content/drive/My Drive/PhD/程式/TrajectoryPrediction/STGAT/STGAT/datasets/zara2/train/crowd

KeyboardInterrupt: ignored

# 程式碼範例

## init用法

In [ ]:
class A():
    def __init__(self, init_age):
        super().__init__()
        print('@__init__() => 我年龄是:',init_age)
        self.age = init_age
 
    def __call__(self, added_age):
        print('@__call__() => :',added_age)
 
        res = self.forward(added_age)
        return res
 
    def forward(self, input_):
        print('@forward() =>  函数被调用了')
        
        return input_ + self.age
print('对象初始化。。。。')
a = A(10)
 
 
input_param = a(2)
print("我现在的年龄是：", input_param)

对象初始化。。。。
@__init__() => 我年龄是: 10
@__call__() => : 2
@forward() =>  函数被调用了
我现在的年龄是： 12


## Super.init()用法

https://zhuanlan.zhihu.com/p/29763421


In [ ]:
__metaclass__ = type  # 想用super就先加上
class A:

    # 帶有兩個下劃線開頭的函數是聲明該屬性為專有，不能在類地外部被使用或直接訪問
    def __init__(self,name='from init',age=0):
        print("#屬於A的_init_()..., name:",name)
        self.name =name
        self.age =age

    def Me1(self,name_Me1='from Me1',age_Me1=1):
        print(self.name,self.age)
        print(name_Me1,age_Me1)

class B(A):

    def __init__(self,name_b='默认的name_b',age_b=5):
        print("#屬於B的_init_()..., name_b:",name_b)
        '''
        這是對繼承自父類的屬性進行初始化。而且是用父類的初始化方法來初始化繼承的屬性。
        也就是說，子類繼承了父類的所有屬性和方法，父類屬性自然會用父類方法來進行初始化。
        '''
        super(B, self).__init__(name='在B类初始化时已修改',age=4)
        self.name_b = name_b
        self.age_b = age_b

    def Me2(self,name_Me2='默认的name_Me2',age_Me2=7):
        print(self.name,self.age)
        print(self.name_b,self.age_b)
        print(name_Me2,age_Me2)



print ('---------------1')
b = B('修改默认值',6)
print ('---------------2')
b.Me1()
print ('---------------3')
b.Me1('修改Me1',2)
print ('---------------4')
b.Me2()
print ('---------------5')
b.Me2('修改Me2',8)
print ('---------------6')
a = A('修改A中init',10)
a.Me1()

---------------1
#屬於B的_init_()..., name_b: 修改默认值
#屬於A的_init_()..., name: 在B类初始化时已修改
---------------2
在B类初始化时已修改 4
from Me1 1
---------------3
在B类初始化时已修改 4
修改Me1 2
---------------4
在B类初始化时已修改 4
修改默认值 6
默认的name_Me2 7
---------------5
在B类初始化时已修改 4
修改默认值 6
修改Me2 8
---------------6
#屬於A的_init_()..., name: 修改A中init
修改A中init 10
from Me1 1


## Dataloader範例

https://morvanzhou.github.io/tutorials/machine-learning/torch/3-05-train-on-batch/